<a href="https://colab.research.google.com/github/VasireddyNandini/predictive-analysis/blob/prod_recommend/product_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from collections import defaultdict
from typing import Set
import heapq

# Sample Data

# Users and their shopping history + interests
users = {
    "U1": {"interests": {"electronics", "gaming"}, "purchases": {"P1", "P2", "P3"}},
    "U2": {"interests": {"fitness", "electronics"}, "purchases": {"P2", "P4"}},
    "U3": {"interests": {"gaming", "books"}, "purchases": {"P3", "P5"}},
    "U4": {"interests": {"gaming", "electronics"}, "purchases": {"P6", "P2", "P3"}},
    "U5": {"interests": {"fitness", "fashion"}, "purchases": {"P7", "P8"}}
}

# Product catalog
products = {
    "P1": {"name": "Wireless Mouse", "category": "electronics"},
    "P2": {"name": "Gaming Keyboard", "category": "gaming"},
    "P3": {"name": "VR Headset", "category": "gaming"},
    "P4": {"name": "Fitness Tracker", "category": "fitness"},
    "P5": {"name": "Gaming Book", "category": "books"},
    "P6": {"name": "Console Controller", "category": "gaming"},
    "P7": {"name": "Yoga Mat", "category": "fitness"},
    "P8": {"name": "Sports Shoes", "category": "fashion"}
}


# Utility: Jaccard similarity
def jaccard_similarity(set1: Set, set2: Set) -> float:
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union else 0


# Recommendation function
def recommend_products(user_id: str, top_n: int = 5):
    if user_id not in users:
        print(f"User ID '{user_id}' not found.\nAvailable users: {', '.join(users.keys())}")
        return []

    target = users[user_id]
    target_purchases = target["purchases"]
    target_interests = target["interests"]

    similarity_scores = []

    for other_user, data in users.items():
        if other_user == user_id:
            continue
        sim_score = jaccard_similarity(target_purchases, data["purchases"])
        interest_score = jaccard_similarity(target_interests, data["interests"])
        total_score = 0.7 * sim_score + 0.3 * interest_score
        if total_score > 0:
            similarity_scores.append((total_score, other_user))

    # Sort users by similarity
    similarity_scores.sort(reverse=True)

    # Collect recommended products
    recommended = defaultdict(float)  # product -> score

    for score, similar_user in similarity_scores:
        for product in users[similar_user]["purchases"]:
            if product not in target_purchases:
                recommended[product] += score

    # Rank and return top N products
    top_products = heapq.nlargest(top_n, recommended.items(), key=lambda x: x[1])

    return [f"{products[pid]['name']} (Score: {round(score, 2)})" for pid, score in top_products]


# 🧑‍💻 Interactive Input
if __name__ == "__main__":
    print("📦 Product Recommendation System")
    print(f"Available users: {', '.join(users.keys())}")
    user_input = input("Enter a user ID to get recommendations (e.g., U1): ").strip()

    result = recommend_products(user_input)

    if result:
        print(f"\n🎯 Top Product Recommendations for {user_input}:\n")
        for idx, item in enumerate(result, 1):
            print(f"{idx}. {item}")


📦 Product Recommendation System
Available users: U1, U2, U3, U4, U5
Enter a user ID to get recommendations (e.g., U1): U2

🎯 Top Product Recommendations for U2:

1. VR Headset (Score: 0.55)
2. Console Controller (Score: 0.27)
3. Wireless Mouse (Score: 0.27)
4. Yoga Mat (Score: 0.1)
5. Sports Shoes (Score: 0.1)
